In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.append('../../pycore/')
from utils import extract_details_from_rxnid

In [2]:
df_enz = pd.read_excel('../../build_GAMS_model/input/ENZYME_stoich_curation_2021-10-01.xlsx')
df_enz.index = df_enz.id.to_list()

In [14]:
df_kcat = pd.read_excel('../input/rxn_enz_kcat_kapp.xlsx')
idx = [df_kcat.rxn_id[i] + '_' + df_kcat.direction[i] for i in df_kcat.index]
df_kcat.index = idx

In [26]:
vitro_0 = pd.core.series.Series(dtype='float64')
df = df_kcat
for i in df.index:
    kcat = df_kcat.loc[i, 'kcat_invitro (1/s)'].max()
    if pd.isnull(kcat) == False:
        vitro_0[i] = kcat

In [28]:
idx = df_enz.index.to_list()
df_write = pd.DataFrame(index=idx, columns=['id', 'kapp (1/s)', 'source'])
df_write['id'] = df_write.index.to_list()
med = round(np.median(vitro_0), 0)

for i in df_enz.index:
    _,rxn,rdir,enz = extract_details_from_rxnid(i)
    if enz in ['SPONT', 'UNKNOWN']:
        df_write.loc[i, 'source'] = enz
        continue
    
    entry = rxn + '_' + rdir
    if entry in vitro_0.index:
        df_write.loc[i, 'kapp (1/s)'] = vitro_0[entry]
        df_write.loc[i, 'source'] = 'parameterization'
    else:
        df_write.loc[i, 'kapp (1/s)'] = med
        df_write.loc[i, 'source'] = 'kapp_median'

In [29]:
df_write.to_excel('PARAMS_kapp_vitro_frompy.xlsx', index=None)